#### Importing Libraries and Packages

In [ ]:
# General utilities and data manipulations

import hashlib
import math
import os
import re
import pandas as pd
import numpy as np
import pytz
import openpyxl
from datetime import datetime
from io import BytesIO
from io import StringIO
import time
# Snowflake and Azure cloud integration
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv
load_dotenv(override=True)

# Configure notebook to display every result in a cell instead of just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Suppress non-critical warnings and future deprecation alerts to keep output clean
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

True

In [58]:
# # Load environment variables from .env file and allow overwriting existing system variables
# load_dotenv(override=True)

# # Retrieve the Azure storage connection string securely from the environment
# CONNECTION_STRING = os.getenv("CONNECTION_STRING")

# # Initialize the Blob Service Client to interact with Azure Blob Storage
# blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING)

# # Define the target container name for viewership and audience data
# CONTAINER_NAME = "easg-followerinsights-demospro"

# #Snowflake Configure
# SNOWFLAKE_USER ='bunnawich.iamsawat@img.com'
# database="FOLLOWERINSIGHTS_ANALYTICS"

In [59]:
CONNECTION_STRING = os.getenv("CONNECTION_STRING")
if CONNECTION_STRING is None:
    raise ValueError("CONNECTION_STRING not found in .env")

CONTAINER_NAME = os.getenv("CONTAINER_NAME", "easg-followerinsights-demospro")
WAREHOUSE = os.getenv("WAREHOUSE", "FOLLOWERINSIGHTS_ANALYTICS")
SNOWFLAKE_ACCOUNT = os.getenv("SNOWFLAKE_ACCOUNT")
SNOWFLAKE_USER = os.getenv("SNOWFLAKE_USER")

blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING)

storage_account_name = "eastordev01"

print(f"Container: {CONTAINER_NAME}")
print(f"Warehouse: {WAREHOUSE}")
print(f"Account: {SNOWFLAKE_ACCOUNT}")
print(f"User: {SNOWFLAKE_USER}")

Container: easg-followerinsights-demospro
Warehouse: FOLLOWERINSIGHTS_ANALYTICS
Account: JCYRTIM-DATASERVICESNONPROD
User: biamsawat@wmegrp.org


In [ ]:
# conn = snowflake.connector.connect(
#     user=SNOWFLAKE_USER,
#     # password=os.getenv('SNOWFLAKE_PASSWORD'),
#     account=os.getenv('SNOWFLAKE_ACCOUNT'),
#     authenticator="externalbrowser",
#     role='ROLE_DATADEVELOPERS_ELEVATED',
#     warehouse=WAREHOUSE,
#     database=WAREHOUSE,
#     schema='BI'
# )





 pip install snowflake-connector-python[secure-local-storage]


Initiating login request with your identity provider. Press CTRL+C to abort and try again...
Going to open: https://login.microsoftonline.com/9c901a7a-a846-4962-94d8-deb98f963564/saml2?SAMLRequest=nZLBctowEIZfxaOeLQvFcWwNkKEwKXSSlAJJO7kJew0aZMmRZAx9%2BgoTZtJDcujNI3%2B7%2BrT%2F9m8PlQz2YKzQaoB6mKAAVK4LoTYD9LS6C1MUWMdVwaVWMEBHsOh22Le8kjUbNW6rFvDagHWBb6Qs634MUGMU09wKyxSvwDKXs%2BXo4Z5RTFhttNO5luhdyecV3FowzhteSgorvN7WuZpFUdu2uL3C2mwiSgiJSBZ56oR8ufAH%2F6YP%2BF5E4hPvCY%2FP39y%2BCnUewWda6zNk2XS1mofzH8sVCkYX1bFWtqnALMHsRQ5Pi%2FuzgPUGu3V8E1OKWz%2B3EBqja8D8T2MAW6XbUvId5LqqG%2Be7Y%2F8VlVBEUm%2BEH8BsMkD1ThTJDmbVYaFkepx%2Fi9e%2FSrIcb78vy9fpy08%2B3Se%2FHx5hEtfp82STo%2BD5kjA9JTyztoGZOuXq%2FBGhSUhoSJIViRm9ZuQGU5q8oGDi%2FYTirqu8yHceuBK50VaXTispFHSWWZ6RHr%2FhIU%2FjJIyzhIZZXKRhAessLbPk6jqJo1PaFJ03iHUiZvi%2Fc%2BlH77u8LeWjz2k2mWsp8mNwp03F3ccx9nCvOxFFWHYog4oLOSoKA9b6OKXU7dgAd373nWkARcPzrf9u%2F%2FAv&RelayState=ver%3A3-hint%3A224923187462166-ETMsDgAAAZwxMlGaABRBRVMvQ0JDL1BLQ1M1UGFkZGluZwEAABAAEDEyOGjqtEM7jo%2B7

 pip install snowflake-connector-python[secure-local-storage]


In [ ]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.serialization import load_pem_private_key
from cryptography.hazmat.backends import default_backend
import snowflake.connector
import os
from dotenv import load_dotenv
load_dotenv(override=True)

sf_private_key_pem = os.getenv("snowflakecredstoken")  # use the same env var as ga script
if not sf_private_key_pem:
    raise ValueError("env var 'snowflakecredstoken' missing")

SF_USER = "svc_user_europe_wme_group_basic"
SF_ACCOUNT = "JCYRTIM-DATASERVICESNONPROD"
SF_ROLE = "ROLE_DATADEVELOPERS_ELEVATED"  # same as ga script
SF_WAREHOUSE = os.getenv("WAREHOUSE", "FOLLOWERINSIGHTS_ANALYTICS")
SF_DATABASE = SF_WAREHOUSE
SF_SCHEMA = "BI"

def get_snowflake_connection():
    private_key = load_pem_private_key(
        sf_private_key_pem.encode(), password=None, backend=default_backend()
    )
    private_key_der = private_key.private_bytes(
        encoding=serialization.Encoding.DER,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption(),
    )
    conn_params = {
        "user": SF_USER,
        "private_key": private_key_der,
        "account": SF_ACCOUNT,
        "warehouse": SF_WAREHOUSE,
        "database": SF_DATABASE,
        "schema": SF_SCHEMA,
        "role": SF_ROLE,
    }
    return snowflake.connector.connect(**conn_params)

conn = get_snowflake_connection()


#### Setting up Directory and Function

##### 1. Paths

###### 1.1 Dropsite

In [61]:
FOLDER_PATH = "dropsite"  # Your dropsite folder
 
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING)
container_client = blob_service_client.get_container_client(CONTAINER_NAME)
 
# List all blobs (files) inside the dropsite folder
blob_list = container_client.list_blobs(name_starts_with=FOLDER_PATH)
 

 
dropsite = []
for blob in blob_list:
    # Only show files (not subdirectories)
    if blob.name.endswith('csv'):
        dropsite.append(blob.name)

dropsite = [file.split('/')[1] for file in dropsite if file.endswith('.csv')]
dropsite


['Andrew Koji_Instagram_@andrewkojigram.csv',
 'Anthony Black_Instagram_@anthonyblack.csv',
 'Beckham Black_Instagram_@beckhamblackk.csv',
 'Luke Combs_Facebook_@LukeCombs.csv',
 'Luke Combs_Instagram_@lukecombs.csv',
 'Luke Combs_TikTok_@lukecombs.csv',
 'Luke Combs_X_@lukecombs.csv',
 'Luke Combs_YouTube_@lukecombs.csv',
 'Yerin Ha_Instagram_@yerinha_.csv',
 'Yerin Ha_Instagram_@yerinha__on_Instagram.csv']

###### 1.2 Data directory, output site and talent keys

In [62]:
# Path generator
def path_gen(CONTAINER_NAME, directory, blob):
  CONTAINER_NAME = CONTAINER_NAME
  BLOB_PATH = f"{directory}/{blob}"

  # create client
  blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING)

  # get blob client
  blob_client = blob_service_client.get_blob_client(
      container=CONTAINER_NAME,
      blob=BLOB_PATH,
  )

  # download blob to memory
  download_stream = blob_client.download_blob()
  blob_bytes = download_stream.readall()
  path = BytesIO(blob_bytes)
  return path



In [63]:
path_directory = path_gen(CONTAINER_NAME, 'data directory', '# Data Directory.xlsx')
# path_outputsite = path_gen('easg-followerinsights-demospro', 'output site',  'Platform Average Stitched.csv')
path_talent_keys = path_gen('easg-keys', '', 'Orchard Key.xlsx')
path_market_keys = path_gen('easg-keys', '', 'Key - Markets.xlsx')

##### 2. Functions

In [64]:
#instead of moving all files, only move selected successful files
def move_raw_storage(CONTAINER_NAME, directory, blob, processed_folder='raw storage'):
    """
    Move successful files from source_path to 'raw storage' in Azure Data Lake Storage Gen2.
    """
    
    # Source and destination paths
    source_blob_path = f"{directory}/{blob}"
    dest_blob_path = f"{processed_folder}/{blob}"
    
    # Create client
    blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING)
    
    # Get source and destination blob clients
    source_blob_client = blob_service_client.get_blob_client(
        container=CONTAINER_NAME,
        blob=source_blob_path
    )
    
    dest_blob_client = blob_service_client.get_blob_client(
        container=CONTAINER_NAME,
        blob=dest_blob_path
    )
    
    try:
        # Copy blob to new location
        dest_blob_client.start_copy_from_url(source_blob_client.url)
        
        # Wait for copy to complete (optional but recommended)
        copy_properties = dest_blob_client.get_blob_properties().copy
        if copy_properties.status == "success":
            # Delete original blob after successful copy
            source_blob_client.delete_blob()
            print(f"Successfully moved {source_blob_path} to {dest_blob_path}")
            return True
        else:
            print(f"Copy status: {copy_properties.status}")
            return False
            
    except Exception as e:
        print(f"Error moving blob: {str(e)}")
        return False

In [65]:
def export_to_adls(df, CONTAINER_NAME, output_folder, filename=None, add_timestamp=True, 
                   file_format='csv', sheet_name='Data'):
    """
    Write DataFrame to ADLS as CSV or Excel with improved timeout handling
    
    Parameters:
    - df: pandas DataFrame to write
    - CONTAINER_NAME: ADLS container name
    - output_folder: folder path in ADLS
    - filename: output filename (optional)
    - add_timestamp: whether to add timestamp to filename
    - file_format: 'csv' or 'excel'
    - sheet_name: Excel sheet name (only used if file_format='excel')
    """
    CONNECTION_STRING = os.getenv("CONNECTION_STRING")
    
    # Generate filename
    if filename is None:
        filename = f"data.{file_format if file_format == 'csv' else 'xlsx'}"
    
    # Ensure correct extension
    if file_format == 'excel':
        if not filename.endswith('.xlsx'):
            filename = filename.replace('.csv', '.xlsx') if '.csv' in filename else f"{filename}.xlsx"
    else:
        if not filename.endswith('.csv'):
            filename = filename.replace('.xlsx', '.csv') if '.xlsx' in filename else f"{filename}.csv"
    
    # Add timestamp
    if add_timestamp:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        ext = '.xlsx' if file_format == 'excel' else '.csv'
        name_without_ext = filename.replace(ext, '')
        filename = f"{name_without_ext}_{timestamp}{ext}"
    
    output_path = f"{output_folder}/{filename}"
    
    # Create blob service client with increased timeout
    blob_service_client = BlobServiceClient.from_connection_string(
        CONNECTION_STRING,
        connection_timeout=600,  # 10 minutes connection timeout
        read_timeout=600  # 10 minutes read timeout
    )
    
    blob_client = blob_service_client.get_blob_client(
        container=CONTAINER_NAME,
        blob=output_path
    )
    
    # Convert to appropriate format
    print(f"Converting DataFrame to {file_format.upper()}...")
    buffer = BytesIO()
    
    if file_format == 'excel':
        df.to_excel(buffer, index=False, sheet_name=sheet_name, engine='openpyxl')
    else:
        df.to_csv(buffer, index=False, header=True)
    
    file_data = buffer.getvalue()
    file_size_mb = len(file_data) / 1024 / 1024
    
    print(f"File size: {file_size_mb:.2f} MB")
    print(f"Uploading to: {output_path}")
    
    # Retry logic with exponential backoff
    max_retries = 3
    for attempt in range(max_retries):
        try:
            # Upload with chunking for large files
            blob_client.upload_blob(
                file_data,
                overwrite=True,
                timeout=900,  # 15 minutes timeout
                max_concurrency=8  # Upload 8 chunks in parallel
            )
            
            print(f"✅ Successfully wrote to: {output_path}")
            
            return output_path
            
        except Exception as e:
            if attempt < max_retries - 1:
                wait_time = 2 ** attempt  # Exponential backoff: 1s, 2s, 4s
                print(f"❌ Upload attempt {attempt + 1} failed: {e}")
                print(f"   Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                print(f"❌ All upload attempts failed after {max_retries} tries")
                raise

In [103]:

def write_df_to_snowflake(
    df,
    database,
    table_name,
    loadmode="overwrite"  # "append" or "overwrite"
):
    """
    Writes a pandas DataFrame to Snowflake using the provided database and table.
    """

    # 1. Connection Check
    if 'conn' not in globals() or conn is None:
        raise RuntimeError("Snowflake connection `conn` is not initialized")

    # 2. Input Validation
    if not table_name or not isinstance(table_name, str):
        raise ValueError("table_name must be a non-empty string")

    if not database or not isinstance(database, str):
        raise ValueError("database must be a non-empty string")

    if df.empty:
        print(f"No data to write to {database}.BI.{table_name}. Skipping.")
        return

    # 3. Preparation
    # Snowflake usually requires uppercase column names
    df.columns = df.columns.map(lambda x: x if ' ' in x or '-' in x else x.upper())
    overwrite = loadmode.lower() == "overwrite"

    # 4. Execution (Variables linked correctly here)
    success, nchunks, nrows, _ = write_pandas(
        conn=conn,
        df=df,
        database=database,    # Linked to function argument
        table_name=table_name, # Linked to function argument
        schema="BI",           # Fixed as per your requirement
        auto_create_table=True,
        overwrite=overwrite,
        use_logical_type = True
    )

    # 5. Result Handling
    if not success:
        raise RuntimeError(f"Failed to write data to {database}.BI.{table_name}")

    print(
        f"✅ Successfully {'overwritten' if overwrite else 'appended'} "
        f"{nrows} rows into {database}.BI.{table_name} ({nchunks} chunk(s))"
    )

In [67]:
# Function prompting users for confirmation
def confirm_check():
    while True:
        response = input("Have you updated Orchard Keys in Teams and uploaded in ADLS?(y/n): ").strip().lower()
        if response == 'y':
#             print("Proceeding with the next steps.")
            return True
        elif response == 'n':
            print("Please complete the checking before proceeding.")
        else:
            print("Invalid response. Please answer 'y' or 'n'.")

##### 3. Lists & Dictionaries

In [68]:
# Setting up Directory File to standardise column names
df_dir = pd.read_excel(path_directory, sheet_name = 'Column Names')

# Generating dictionary to clean column names
old_cols = df_dir['Undesired'].tolist()
new_cols = df_dir['Desired'].tolist()

cols_clean = dict(zip(old_cols, new_cols))

In [69]:
# Setting up Directory File to standardise social media platform names

df_dir1 = pd.read_excel(path_directory, sheet_name = 'Social Media Platforms')
# Generating dictionary to clean items

old_cols1 = df_dir1['Undesired'].tolist()
new_cols1 = df_dir1['Desired'].tolist()

cols_clean1 = dict(zip(old_cols1, new_cols1))

In [70]:
# Setting up Directory File to standardise social media platform URLS

df_dir2 = pd.read_excel(path_directory, sheet_name = 'Social Media URL')

# Generating dictionary to clean itemss

old_cols2 = df_dir2['Undesired'].tolist()
new_cols2 = df_dir2['Desired'].tolist()

cols_clean2 = dict(zip(old_cols2, new_cols2))

In [71]:
# Setting up Directory File to standardize entity names
talent_key = pd.read_excel(
    path_talent_keys,
    sheet_name='KEY',
    usecols=['Source', 'Undesired', 'Desired','DP Keep'],
    dtype={'Undesired': str, 'Desired': str}
)

# Apply title casing to the 'Source' column
talent_key['Source'] = talent_key['Source'].str.title()

# Filter to respective Source
talent_key = talent_key[talent_key['Source'] == "Demospro"]  # Change accordingly

# Trim whitespace in all columns
talent_key = talent_key.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Drop duplicates
talent_key.drop_duplicates(keep="last", inplace=True)

# Create dictionary to standardize entity names
clean_talent = dict(zip(talent_key["Undesired"], talent_key["Desired"]))

In [72]:
market_key = pd.read_excel(
    path_market_keys,
    sheet_name='Regions and Markets',
    usecols=['Market', 'Region'],
    dtype={'Market': str, 'Region': str}
)

In [73]:
market_key_V2 = pd.read_excel(
    path_market_keys,
    sheet_name='Key - Markets',
    usecols=['Undesired', 'Desired'],
    dtype={'Undesired': str, 'Desired': str}
)

##### 4. Variables

In [74]:
# Create a timezone object for Singapore
sgt = pytz.timezone('Asia/Singapore')
now = datetime.now(sgt)
time_str = now.strftime('%Y-%m-%d %H:%M:%S')

#### Accessing Files in Dropsite

In [75]:
raw_files = []

for file in dropsite:
    try:
        # Accessing files in dropsite folder in ADLS
        df = pd.read_csv(
            path_gen('easg-followerinsights-demospro', 'dropsite', file), 
            skiprows=2)

        # Change column names
        df.rename(cols_clean, axis=1, inplace=True)

        # Extract the filename without extension and split it
        file_name_parts = os.path.splitext(os.path.basename(file))[0].split('_')

        # Extract talent name
        Entity_name = file_name_parts[0].strip()
        df["Entity"] = Entity_name

        # Extract social media platform
        platform = file_name_parts[1].lower().strip()
        df["Platform"] = platform

        if platform not in old_cols1:
            raise ValueError(f"Platform '{platform}' in file '{file}' is not in the list of acceptable platforms.")

        # Extract Social Media handle (joining the remaining parts)
        social_media_handle = "_".join(file_name_parts[2:]).strip()
        df["Social Media Handle"] = social_media_handle

        if '@' not in social_media_handle:
            raise ValueError(f"Social media handle '{social_media_handle}' in file '{file}' does not contain '@'.")

        # Add in date-time so that we can track when we add in a talent
        df["Date"] = time_str

        # Stitch
        raw_files.append(df)
        
        # Move the processed raw file to the 'raw storage' folder (files with error will remain in dropsite)
        move_raw_storage(CONTAINER_NAME, 'dropsite', file)

    except Exception as e:
        print(f"Error processing file {file}: {e}")

# Combine all processed dataframes
df_join = pd.concat(raw_files, ignore_index=True)
df_join


Successfully moved dropsite/Andrew Koji_Instagram_@andrewkojigram.csv to raw storage/Andrew Koji_Instagram_@andrewkojigram.csv


True

Successfully moved dropsite/Anthony Black_Instagram_@anthonyblack.csv to raw storage/Anthony Black_Instagram_@anthonyblack.csv


True

Successfully moved dropsite/Beckham Black_Instagram_@beckhamblackk.csv to raw storage/Beckham Black_Instagram_@beckhamblackk.csv


...

Successfully moved dropsite/Yerin Ha_Instagram_@yerinha_.csv to raw storage/Yerin Ha_Instagram_@yerinha_.csv


True

Successfully moved dropsite/Yerin Ha_Instagram_@yerinha__on_Instagram.csv to raw storage/Yerin Ha_Instagram_@yerinha__on_Instagram.csv


True

,Category,Question,Response,Number in Sample,No. of Followers,% of Followers,Platform Average,Range From,Range To,Entity,Platform,Social Media Handle,Date
0,BASICS,Gender,Male,364,55168,63.85965,44.503458,3.769046,95.016741,Andrew Koji,instagram,@andrewkojigram,2026-02-06 12:25:24
1,BASICS,Gender,Female,206,31221,36.14035,55.496542,4.983259,96.230954,Andrew Koji,instagram,@andrewkojigram,2026-02-06 12:25:24
2,BASICS,Family status,Single,36,39367,45.56962,50.570346,0.000000,100.000000,Andrew Koji,instagram,@andrewkojigram,2026-02-06 12:25:24
3,BASICS,Family status,Married,43,47022,54.43038,49.339654,0.000000,100.000000,Andrew Koji,instagram,@andrewkojigram,2026-02-06 12:25:24
4,BASICS,Family status,Parents,84,11574,13.39713,7.477016,0.069493,43.474388,Andrew Koji,instagram,@andrewkojigram,2026-02-06 12:25:24
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33297,BRANDS,Brands,Airbnb,79,17422,3.03147,2.866785,0.000000,16.753537,Yerin Ha,instagram,@yerinha__on_Instagram,2026-02-06 12:25:24
33298,BRANDS,Brands,Oreo Cookie,55,12130,2.11051,1.872967,0.000000,11.238671,Yerin Ha,instagram,@yerinha__on_Instagram,2026-02-06 12:25:24
33299,BRANDS,Brands,Vans,176,38814,6.75365,7.972068,0.281492,46.233649,Yerin Ha,instagram,@yerinha__on_Instagram,2026-02-06 12:25:24
33300,BRANDS,Brands,FashionNova,145,31978,5.56408,7.191489,0.139762,48.138298,Yerin Ha,instagram,@yerinha__on_Instagram,2026-02-06 12:25:24


#### Cleaning Data from Dropsite

In [76]:
#Drop Columns
df_join = df_join.drop(columns=["Number in Sample", "Range From", "Range To"])

#Creating "Cat-Qns-Response" column acting as ID
df_join["Cat-Qns-Response"] = df_join ["Category"] + " - " + df_join ["Question"] + " - " + df_join ["Response"]

#Calculation
## Define the columns that need to be divided by 100 and formatted as percentages
columns_to_convert = ["% of Followers", "Platform Average"]

## Convert the columns to percentages by dividing by 100 and format them
df_join[columns_to_convert] = df_join[columns_to_convert].apply(lambda x: x / 100)

#change Social Media platforms to proper names according to Key
df_join["Platform"] = df_join["Platform"].map(cols_clean1)

# Add Social Media URL based on platform 
df_join["Social Media URL"] = df_join["Platform"].map(cols_clean2)

#Remove @ from Social Media Handle Column
df_join["Social Media Handle"] = df_join["Social Media Handle"].str[1:]

#Attached URL with social media handle
df_join["Social Media URL"] = df_join["Social Media URL"] + df_join["Social Media Handle"]

In [77]:
# Find missing entities in KEY
desired_entities = talent_key['Desired'].unique()
undesired_entities = talent_key['Undesired'].unique()
missing_entities = df_join[~df_join['Entity'].isin(desired_entities) & ~df_join['Entity'].isin(undesired_entities)]['Entity'].unique()

# Create DataFrame of missing entities
check = pd.DataFrame({'Latest Missing Entities in Key (Please update them before proceeding)': missing_entities})
# print("No. of Missing Entities:",len(check))
check

,Latest Missing Entities in Key (Please update them before proceeding)


In [78]:
# Call the function and act based on the response
if confirm_check():
    print("Proceeding with next steps.")

Proceeding with next steps.


In [79]:
# Setting up Directory File to standardise entity names
talent_key= pd.read_excel(path_talent_keys, sheet_name = 'KEY', dtype={'Undesired': str, 'Desired': str} )

#Filter to respective Source
talent_key= talent_key[talent_key['Source'] == "Demospro"] #Change accordingly
talent_key.drop_duplicates(keep = "last", inplace=True)
clean_talent = dict(zip(talent_key["Undesired"], talent_key["Desired"]))

In [80]:
# Clean Entity Names using Talent Key
df_join["Entity"] = df_join["Entity"].replace(clean_talent)

In [81]:
# Creating "Entity - Platform" column
social_dict={
    
    "Instagram" : "IG",
    "X"         : "X",
    "Facebook"  : "FB",
    "YouTube"   : "YT",
    "TikTok"    : "TT"   
}

df_join["Platform ID"] =  df_join["Platform"].replace(social_dict)

df_join["Entity - Platform"] =  df_join["Entity"] + " - " + df_join["Platform ID"]

In [82]:
# Add "Is Dup" column with value "No" for original rows
df_join['Is Dup'] = 'No'

In [83]:
# Merge df_join with talent_key on the name columns
df_join = df_join.merge(
    talent_key[['Desired', 'DP Keep']],  # only keep relevant columns
    left_on='Entity',
    right_on='Desired',
    how='left'  # keep all rows from df_join
)

# Rename the DP Keep column to Keep
df_join['Keep'] = df_join['DP Keep']

# Optionally, drop the extra 'Desired' and 'DP Keep' columns
df_join = df_join.drop(columns=['Desired', 'DP Keep'])

df_join['Keep'] = df_join['Keep'].replace({None: np.nan})


In [84]:
df_join['Entity'].unique()


array(['Andrew Koji', 'Anthony Black', 'Beckham Black', 'Luke Combs',
       'Yerin Ha'], dtype=object)

In [85]:
df_join

,Category,Question,Response,No. of Followers,% of Followers,Platform Average,Entity,Platform,Social Media Handle,Date,Cat-Qns-Response,Social Media URL,Platform ID,Entity - Platform,Is Dup,Keep
0,BASICS,Gender,Male,55168,0.638597,0.445035,Andrew Koji,Instagram,andrewkojigram,2026-02-06 12:25:24,BASICS - Gender - Male,https://www.instagram.com/andrewkojigram,IG,Andrew Koji - IG,No,NaN
1,BASICS,Gender,Female,31221,0.361403,0.554965,Andrew Koji,Instagram,andrewkojigram,2026-02-06 12:25:24,BASICS - Gender - Female,https://www.instagram.com/andrewkojigram,IG,Andrew Koji - IG,No,NaN
2,BASICS,Family status,Single,39367,0.455696,0.505703,Andrew Koji,Instagram,andrewkojigram,2026-02-06 12:25:24,BASICS - Family status - Single,https://www.instagram.com/andrewkojigram,IG,Andrew Koji - IG,No,NaN
3,BASICS,Family status,Married,47022,0.544304,0.493397,Andrew Koji,Instagram,andrewkojigram,2026-02-06 12:25:24,BASICS - Family status - Married,https://www.instagram.com/andrewkojigram,IG,Andrew Koji - IG,No,NaN
4,BASICS,Family status,Parents,11574,0.133971,0.074770,Andrew Koji,Instagram,andrewkojigram,2026-02-06 12:25:24,BASICS - Family status - Parents,https://www.instagram.com/andrewkojigram,IG,Andrew Koji - IG,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33297,BRANDS,Brands,Airbnb,17422,0.030315,0.028668,Yerin Ha,Instagram,yerinha__on_Instagram,2026-02-06 12:25:24,BRANDS - Brands - Airbnb,https://www.instagram.com/yerinha__on_Instagram,IG,Yerin Ha - IG,No,NaN
33298,BRANDS,Brands,Oreo Cookie,12130,0.021105,0.018730,Yerin Ha,Instagram,yerinha__on_Instagram,2026-02-06 12:25:24,BRANDS - Brands - Oreo Cookie,https://www.instagram.com/yerinha__on_Instagram,IG,Yerin Ha - IG,No,NaN
33299,BRANDS,Brands,Vans,38814,0.067536,0.079721,Yerin Ha,Instagram,yerinha__on_Instagram,2026-02-06 12:25:24,BRANDS - Brands - Vans,https://www.instagram.com/yerinha__on_Instagram,IG,Yerin Ha - IG,No,NaN
33300,BRANDS,Brands,FashionNova,31978,0.055641,0.071915,Yerin Ha,Instagram,yerinha__on_Instagram,2026-02-06 12:25:24,BRANDS - Brands - FashionNova,https://www.instagram.com/yerinha__on_Instagram,IG,Yerin Ha - IG,No,NaN


##### 1. Creating a duplicated row for each talent and question to benchmark against platform average on Power BI Chart Visual

In [86]:
df_filtered = df_join.copy()

# Modify the duplicated rows.move the plat avg values to % of followers so that they are the same column for the duped values
df_filtered['% of Followers'] = df_filtered['Platform Average']
df_filtered['No. of Followers'] = 0
df_filtered["Entity - Platform"] = df_filtered["Platform ID"] + " - Average"
df_filtered['Is Dup'] = 'Yes' 

# Concatenate the original and duplicated DataFrames
df_join = pd.concat([df_join, df_filtered])
df_join

,Category,Question,Response,No. of Followers,% of Followers,Platform Average,Entity,Platform,Social Media Handle,Date,Cat-Qns-Response,Social Media URL,Platform ID,Entity - Platform,Is Dup,Keep
0,BASICS,Gender,Male,55168,0.638597,0.445035,Andrew Koji,Instagram,andrewkojigram,2026-02-06 12:25:24,BASICS - Gender - Male,https://www.instagram.com/andrewkojigram,IG,Andrew Koji - IG,No,NaN
1,BASICS,Gender,Female,31221,0.361403,0.554965,Andrew Koji,Instagram,andrewkojigram,2026-02-06 12:25:24,BASICS - Gender - Female,https://www.instagram.com/andrewkojigram,IG,Andrew Koji - IG,No,NaN
2,BASICS,Family status,Single,39367,0.455696,0.505703,Andrew Koji,Instagram,andrewkojigram,2026-02-06 12:25:24,BASICS - Family status - Single,https://www.instagram.com/andrewkojigram,IG,Andrew Koji - IG,No,NaN
3,BASICS,Family status,Married,47022,0.544304,0.493397,Andrew Koji,Instagram,andrewkojigram,2026-02-06 12:25:24,BASICS - Family status - Married,https://www.instagram.com/andrewkojigram,IG,Andrew Koji - IG,No,NaN
4,BASICS,Family status,Parents,11574,0.133971,0.074770,Andrew Koji,Instagram,andrewkojigram,2026-02-06 12:25:24,BASICS - Family status - Parents,https://www.instagram.com/andrewkojigram,IG,Andrew Koji - IG,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33297,BRANDS,Brands,Airbnb,0,0.028668,0.028668,Yerin Ha,Instagram,yerinha__on_Instagram,2026-02-06 12:25:24,BRANDS - Brands - Airbnb,https://www.instagram.com/yerinha__on_Instagram,IG,IG - Average,Yes,NaN
33298,BRANDS,Brands,Oreo Cookie,0,0.018730,0.018730,Yerin Ha,Instagram,yerinha__on_Instagram,2026-02-06 12:25:24,BRANDS - Brands - Oreo Cookie,https://www.instagram.com/yerinha__on_Instagram,IG,IG - Average,Yes,NaN
33299,BRANDS,Brands,Vans,0,0.079721,0.079721,Yerin Ha,Instagram,yerinha__on_Instagram,2026-02-06 12:25:24,BRANDS - Brands - Vans,https://www.instagram.com/yerinha__on_Instagram,IG,IG - Average,Yes,NaN
33300,BRANDS,Brands,FashionNova,0,0.071915,0.071915,Yerin Ha,Instagram,yerinha__on_Instagram,2026-02-06 12:25:24,BRANDS - Brands - FashionNova,https://www.instagram.com/yerinha__on_Instagram,IG,IG - Average,Yes,NaN


##### 2. Extract latest platform average and update values in "output site" file

In [87]:
platform_average =df_join[["Category", "Question", "Response","Cat-Qns-Response", "Platform Average", "Platform", "Date", "Platform ID"]]

In [88]:
#adding additional columns to match main table
platform_average["Entity"] = "Platform Average"
platform_average["Entity - Platform"] = platform_average["Platform ID"] + " - Average"
platform_average["% of Followers"] = platform_average["Platform Average"]
platform_average['Is Dup'] = 'No'
platform_average["No. of Followers"] = np.nan
platform_average["Social Media Handle"] = np.nan
platform_average["Social Media URL"] = np.nan

In [89]:
desired_column_order = [
    'Category', 
    'Question', 
    'Response', 
    'Cat-Qns-Response', 
    'Entity', 
    'No. of Followers', 
    '% of Followers', 
    'Platform Average', 
    'Platform', 
    'Social Media Handle', 
    'Social Media URL', 
    'Date', 
    'Entity - Platform', 
    'Platform ID', 
    'Is Dup'
]

platform_average = platform_average[desired_column_order]

platform_average.fillna("-", inplace=True)
platform_average

,Category,Question,Response,Cat-Qns-Response,Entity,No. of Followers,% of Followers,Platform Average,Platform,Social Media Handle,Social Media URL,Date,Entity - Platform,Platform ID,Is Dup
0,BASICS,Gender,Male,BASICS - Gender - Male,Platform Average,-,0.445035,0.445035,Instagram,-,-,2026-02-06 12:25:24,IG - Average,IG,No
1,BASICS,Gender,Female,BASICS - Gender - Female,Platform Average,-,0.554965,0.554965,Instagram,-,-,2026-02-06 12:25:24,IG - Average,IG,No
2,BASICS,Family status,Single,BASICS - Family status - Single,Platform Average,-,0.505703,0.505703,Instagram,-,-,2026-02-06 12:25:24,IG - Average,IG,No
3,BASICS,Family status,Married,BASICS - Family status - Married,Platform Average,-,0.493397,0.493397,Instagram,-,-,2026-02-06 12:25:24,IG - Average,IG,No
4,BASICS,Family status,Parents,BASICS - Family status - Parents,Platform Average,-,0.074770,0.074770,Instagram,-,-,2026-02-06 12:25:24,IG - Average,IG,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33297,BRANDS,Brands,Airbnb,BRANDS - Brands - Airbnb,Platform Average,-,0.028668,0.028668,Instagram,-,-,2026-02-06 12:25:24,IG - Average,IG,No
33298,BRANDS,Brands,Oreo Cookie,BRANDS - Brands - Oreo Cookie,Platform Average,-,0.018730,0.018730,Instagram,-,-,2026-02-06 12:25:24,IG - Average,IG,No
33299,BRANDS,Brands,Vans,BRANDS - Brands - Vans,Platform Average,-,0.079721,0.079721,Instagram,-,-,2026-02-06 12:25:24,IG - Average,IG,No
33300,BRANDS,Brands,FashionNova,BRANDS - Brands - FashionNova,Platform Average,-,0.071915,0.071915,Instagram,-,-,2026-02-06 12:25:24,IG - Average,IG,No


In [90]:
display(platform_average)

,Category,Question,Response,Cat-Qns-Response,Entity,No. of Followers,% of Followers,Platform Average,Platform,Social Media Handle,Social Media URL,Date,Entity - Platform,Platform ID,Is Dup
0,BASICS,Gender,Male,BASICS - Gender - Male,Platform Average,-,0.445035,0.445035,Instagram,-,-,2026-02-06 12:25:24,IG - Average,IG,No
1,BASICS,Gender,Female,BASICS - Gender - Female,Platform Average,-,0.554965,0.554965,Instagram,-,-,2026-02-06 12:25:24,IG - Average,IG,No
2,BASICS,Family status,Single,BASICS - Family status - Single,Platform Average,-,0.505703,0.505703,Instagram,-,-,2026-02-06 12:25:24,IG - Average,IG,No
3,BASICS,Family status,Married,BASICS - Family status - Married,Platform Average,-,0.493397,0.493397,Instagram,-,-,2026-02-06 12:25:24,IG - Average,IG,No
4,BASICS,Family status,Parents,BASICS - Family status - Parents,Platform Average,-,0.074770,0.074770,Instagram,-,-,2026-02-06 12:25:24,IG - Average,IG,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33297,BRANDS,Brands,Airbnb,BRANDS - Brands - Airbnb,Platform Average,-,0.028668,0.028668,Instagram,-,-,2026-02-06 12:25:24,IG - Average,IG,No
33298,BRANDS,Brands,Oreo Cookie,BRANDS - Brands - Oreo Cookie,Platform Average,-,0.018730,0.018730,Instagram,-,-,2026-02-06 12:25:24,IG - Average,IG,No
33299,BRANDS,Brands,Vans,BRANDS - Brands - Vans,Platform Average,-,0.079721,0.079721,Instagram,-,-,2026-02-06 12:25:24,IG - Average,IG,No
33300,BRANDS,Brands,FashionNova,BRANDS - Brands - FashionNova,Platform Average,-,0.071915,0.071915,Instagram,-,-,2026-02-06 12:25:24,IG - Average,IG,No


In [91]:
path_platform_average = path_gen(CONTAINER_NAME, 'output site', 'Platform Average Stitched (VSCODE).csv')
current_platform_avg = pd.read_csv(path_platform_average)

In [92]:
current_platform_avg

,Category,Question,Response,Cat-Qns-Response,Entity,No. of Followers,% of Followers,Platform Average,Platform,Social Media Handle,Social Media URL,Date,Entity - Platform,Platform ID,Is Dup
0,BASICS,Native language,English,BASICS - Native language - English,Platform Average,-,0.756283,0.756283,Facebook,-,-,2026-02-06 12:02:14,FB - Average,FB,No
1,BASICS,Race (US only),White/caucasian,BASICS - Race (US only) - White/caucasian,Platform Average,-,0.755311,0.755311,X,-,-,2026-02-06 12:02:14,X - Average,X,No
2,BASICS,Race (US only),White/caucasian,BASICS - Race (US only) - White/caucasian,Platform Average,-,0.750711,0.750711,Facebook,-,-,2026-02-06 12:02:14,FB - Average,FB,No
3,BASICS,Race (US only),White/caucasian,BASICS - Race (US only) - White/caucasian,Platform Average,-,0.743830,0.743830,YouTube,-,-,2026-02-06 12:02:14,YT - Average,YT,No
4,BASICS,Family status,Single,BASICS - Family status - Single,Platform Average,-,0.708375,0.708375,YouTube,-,-,2026-02-06 12:02:14,YT - Average,YT,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435813,WORK,Employer,Ashland University,WORK - Employer - Ashland University,Platform Average,0.0,0.000000,0.000000,TikTok,-,-,2024-07-08_10:11:42,TT - Average,TT,No
435814,WORK,Employer,Books A Million,WORK - Employer - Books A Million,Platform Average,0.0,0.000000,0.000000,TikTok,-,-,2024-07-08_10:11:42,TT - Average,TT,No
435815,WORK,Employer,Camping World and Good Sam,WORK - Employer - Camping World and Good Sam,Platform Average,0.0,0.000000,0.000000,YouTube,-,-,2024-07-08_10:11:42,YT - Average,YT,No
435816,BRANDS,Brands,Branston,BRANDS - Brands - Branston,Platform Average,0.0,0.000000,0.000000,TikTok,-,-,2024-07-08_10:11:42,TT - Average,TT,No


In [93]:
# #Read Plaform Average Stitched from Output site
# FOLDER_PATH = "output site" 
 
# blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING)
# container_client = blob_service_client.get_container_client(CONTAINER_NAME)
 
# # List all blobs (files) inside the output folder
# blob_list = container_client.list_blobs(name_starts_with=FOLDER_PATH)
 

 
# dfs = []
# for blob in blob_list:
#     if "part-" in blob.name:
#         blob_client = container_client.get_blob_client(blob)
#         data = blob_client.download_blob().readall()
#         df_part = pd.read_csv(BytesIO(data), engine="python", on_bad_lines="skip")
#         dfs.append(df_part)
 
# # merge part files
# current_platform_avg = pd.concat(dfs, ignore_index=True)


In [94]:
# current_platform_avg.shape

In [95]:
#Concatenate existing platform average and new platform average
stitched_platform_average = pd.concat([current_platform_avg , platform_average])

# Sort by "Cat-Qns-Response" and "Platform ID", then by "Platform Average" in descending order
stitched_platform_average = stitched_platform_average.sort_values(by=["Date", "Platform Average"], ascending=[False, False])

stitched_platform_average

# Drop duplicates considering all columns except "Date", keeping the first occurrence (highest "Platform Average")
columns_to_consider = ["Cat-Qns-Response", "Platform ID"]
print(f"Before duplicate removal, length: {len(stitched_platform_average)}")
stitched_platform_average = stitched_platform_average.drop_duplicates(subset=columns_to_consider, keep='first')
print(f"After duplicate removal, length: {len(stitched_platform_average)}")

stitched_platform_average["% of Followers"] = stitched_platform_average["Platform Average"]

# Reset the index of the resulting DataFrame
stitched_platform_average = stitched_platform_average.reset_index(drop=True)

# Display the resulting DataFrame
stitched_platform_average['No. of Followers'] = stitched_platform_average['No. of Followers'].astype(str)

stitched_platform_average


,Category,Question,Response,Cat-Qns-Response,Entity,No. of Followers,% of Followers,Platform Average,Platform,Social Media Handle,Social Media URL,Date,Entity - Platform,Platform ID,Is Dup
7544,BASICS,Native language,English,BASICS - Native language - English,Platform Average,-,0.756283,0.756283,Facebook,-,-,2026-02-06 12:25:24,FB - Average,FB,No
7544,BASICS,Native language,English,BASICS - Native language - English,Platform Average,-,0.756283,0.756283,Facebook,-,-,2026-02-06 12:25:24,FB - Average,FB,No
18828,BASICS,Race (US only),White/caucasian,BASICS - Race (US only) - White/caucasian,Platform Average,-,0.755311,0.755311,X,-,-,2026-02-06 12:25:24,X - Average,X,No
18828,BASICS,Race (US only),White/caucasian,BASICS - Race (US only) - White/caucasian,Platform Average,-,0.755311,0.755311,X,-,-,2026-02-06 12:25:24,X - Average,X,No
7540,BASICS,Race (US only),White/caucasian,BASICS - Race (US only) - White/caucasian,Platform Average,-,0.750711,0.750711,Facebook,-,-,2026-02-06 12:25:24,FB - Average,FB,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435813,WORK,Employer,Ashland University,WORK - Employer - Ashland University,Platform Average,0.0,0.000000,0.000000,TikTok,-,-,2024-07-08_10:11:42,TT - Average,TT,No
435814,WORK,Employer,Books A Million,WORK - Employer - Books A Million,Platform Average,0.0,0.000000,0.000000,TikTok,-,-,2024-07-08_10:11:42,TT - Average,TT,No
435815,WORK,Employer,Camping World and Good Sam,WORK - Employer - Camping World and Good Sam,Platform Average,0.0,0.000000,0.000000,YouTube,-,-,2024-07-08_10:11:42,YT - Average,YT,No
435816,BRANDS,Brands,Branston,BRANDS - Brands - Branston,Platform Average,0.0,0.000000,0.000000,TikTok,-,-,2024-07-08_10:11:42,TT - Average,TT,No


Before duplicate removal, length: 502422
After duplicate removal, length: 435818


,Category,Question,Response,Cat-Qns-Response,Entity,No. of Followers,% of Followers,Platform Average,Platform,Social Media Handle,Social Media URL,Date,Entity - Platform,Platform ID,Is Dup
0,BASICS,Native language,English,BASICS - Native language - English,Platform Average,-,0.756283,0.756283,Facebook,-,-,2026-02-06 12:25:24,FB - Average,FB,No
1,BASICS,Race (US only),White/caucasian,BASICS - Race (US only) - White/caucasian,Platform Average,-,0.755311,0.755311,X,-,-,2026-02-06 12:25:24,X - Average,X,No
2,BASICS,Race (US only),White/caucasian,BASICS - Race (US only) - White/caucasian,Platform Average,-,0.750711,0.750711,Facebook,-,-,2026-02-06 12:25:24,FB - Average,FB,No
3,BASICS,Race (US only),White/caucasian,BASICS - Race (US only) - White/caucasian,Platform Average,-,0.743830,0.743830,YouTube,-,-,2026-02-06 12:25:24,YT - Average,YT,No
4,BASICS,Family status,Single,BASICS - Family status - Single,Platform Average,-,0.708375,0.708375,YouTube,-,-,2026-02-06 12:25:24,YT - Average,YT,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435813,WORK,Employer,Ashland University,WORK - Employer - Ashland University,Platform Average,0.0,0.000000,0.000000,TikTok,-,-,2024-07-08_10:11:42,TT - Average,TT,No
435814,WORK,Employer,Books A Million,WORK - Employer - Books A Million,Platform Average,0.0,0.000000,0.000000,TikTok,-,-,2024-07-08_10:11:42,TT - Average,TT,No
435815,WORK,Employer,Camping World and Good Sam,WORK - Employer - Camping World and Good Sam,Platform Average,0.0,0.000000,0.000000,YouTube,-,-,2024-07-08_10:11:42,YT - Average,YT,No
435816,BRANDS,Brands,Branston,BRANDS - Brands - Branston,Platform Average,0.0,0.000000,0.000000,TikTok,-,-,2024-07-08_10:11:42,TT - Average,TT,No


In [96]:
# Step 1: Keep only rows where Question == "Location: by country"
filtered_df = stitched_platform_average[stitched_platform_average['Question'] == 'Location: by country']

# Step 2: Drop duplicates based on Cat-Qns-Response and Platform
deduped_df = filtered_df.drop_duplicates(subset=['Cat-Qns-Response', 'Platform'])

# Step 3: Select only the required columns and assign to final DataFrame
platform_average_region = deduped_df[['Platform', 'Response','Platform Average']]
platform_average_region = platform_average_region.rename(columns={'Platform Average': 'Platform Average Market'})

platform_average_region

,Platform,Response,Platform Average Market
42,YouTube,United States,0.383954
43,Facebook,United States,0.383422
54,X,United States,0.335113
65,Instagram,United States,0.313134
222,TikTok,United States,0.176015
...,...,...,...
406457,X,Pitcairn,0.000000
407628,Facebook,Vatican,0.000002
417508,Instagram,BenÃ­nâ€‹,0.000039
428378,TikTok,Serbia and Montenegro,0.000007


In [97]:
# Perform left join on Response = Market to get Region
market_key_final = platform_average_region.merge(
    market_key[['Region', 'Market']], 
    how='left',
    left_on='Response',
    right_on='Market'
)

# Calculate the average Platform Average Market per Region and Platform
market_key_final['Platform Average Region'] = (
    market_key_final.groupby(['Region', 'Platform'])['Platform Average Market']
    .transform('sum')
)
# Drop the redundant 'Market' column
market_key_final = market_key_final.drop(columns=['Response', 'Platform Average Market'])

In [98]:
# arrange order
desired_column_order = [
    'Category', 
    'Question', 
    'Response', 
    'Cat-Qns-Response', 
    'Entity', 
    'No. of Followers', 
    '% of Followers', 
    'Platform Average', 
    'Platform', 
    'Social Media Handle', 
    'Social Media URL', 
    'Date', 
    'Entity - Platform', 
    'Platform ID', 
    'Is Dup',
    'Keep'
]

# Reorder the DataFrame
df_join = df_join[desired_column_order]
df_join

,Category,Question,Response,Cat-Qns-Response,Entity,No. of Followers,% of Followers,Platform Average,Platform,Social Media Handle,Social Media URL,Date,Entity - Platform,Platform ID,Is Dup,Keep
0,BASICS,Gender,Male,BASICS - Gender - Male,Andrew Koji,55168,0.638597,0.445035,Instagram,andrewkojigram,https://www.instagram.com/andrewkojigram,2026-02-06 12:25:24,Andrew Koji - IG,IG,No,NaN
1,BASICS,Gender,Female,BASICS - Gender - Female,Andrew Koji,31221,0.361403,0.554965,Instagram,andrewkojigram,https://www.instagram.com/andrewkojigram,2026-02-06 12:25:24,Andrew Koji - IG,IG,No,NaN
2,BASICS,Family status,Single,BASICS - Family status - Single,Andrew Koji,39367,0.455696,0.505703,Instagram,andrewkojigram,https://www.instagram.com/andrewkojigram,2026-02-06 12:25:24,Andrew Koji - IG,IG,No,NaN
3,BASICS,Family status,Married,BASICS - Family status - Married,Andrew Koji,47022,0.544304,0.493397,Instagram,andrewkojigram,https://www.instagram.com/andrewkojigram,2026-02-06 12:25:24,Andrew Koji - IG,IG,No,NaN
4,BASICS,Family status,Parents,BASICS - Family status - Parents,Andrew Koji,11574,0.133971,0.074770,Instagram,andrewkojigram,https://www.instagram.com/andrewkojigram,2026-02-06 12:25:24,Andrew Koji - IG,IG,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33297,BRANDS,Brands,Airbnb,BRANDS - Brands - Airbnb,Yerin Ha,0,0.028668,0.028668,Instagram,yerinha__on_Instagram,https://www.instagram.com/yerinha__on_Instagram,2026-02-06 12:25:24,IG - Average,IG,Yes,NaN
33298,BRANDS,Brands,Oreo Cookie,BRANDS - Brands - Oreo Cookie,Yerin Ha,0,0.018730,0.018730,Instagram,yerinha__on_Instagram,https://www.instagram.com/yerinha__on_Instagram,2026-02-06 12:25:24,IG - Average,IG,Yes,NaN
33299,BRANDS,Brands,Vans,BRANDS - Brands - Vans,Yerin Ha,0,0.079721,0.079721,Instagram,yerinha__on_Instagram,https://www.instagram.com/yerinha__on_Instagram,2026-02-06 12:25:24,IG - Average,IG,Yes,NaN
33300,BRANDS,Brands,FashionNova,BRANDS - Brands - FashionNova,Yerin Ha,0,0.071915,0.071915,Instagram,yerinha__on_Instagram,https://www.instagram.com/yerinha__on_Instagram,2026-02-06 12:25:24,IG - Average,IG,Yes,NaN


#### Export

##### Export to ADLS folder

###### Update Platform Values

In [ ]:
# Export CSV data proc
export_to_adls(
    df=stitched_platform_average, # Change to respective dataframe
    CONTAINER_NAME=CONTAINER_NAME, # Change to respective container name
    output_folder="output site", # Change to respective folder path
    filename="Platform Average Stitched (VSCODE)", # Input file name
    file_format='csv', # Input file format, 'csv' or 'excel'
    add_timestamp=False # For proc storage = True, for staging storage =  False
) 

Converting DataFrame to CSV...
File size: 70.11 MB
Uploading to: output site/Platform Average Stitched (VSCODE).csv
✅ Successfully wrote to: output site/Platform Average Stitched (VSCODE).csv


'output site/Platform Average Stitched (VSCODE).csv'

##### Export to Snowflake


###### 1. Main Table

In [ ]:
# write_df_to_snowflake(
#     df=df_join, 
#     database=WAREHOUSE, 
#     table_name='FOLLOWER_INSIGHTS', 
#     loadmode="append"
# ) hidden

✅ Successfully appended 66604 rows into FOLLOWERINSIGHTS_ANALYTICS.BI.FOLLOWER_INSIGHTS (1 chunk(s))



###### 2. Platform Average

In [ ]:
write_df_to_snowflake(
    df=stitched_platform_average, 
    database=WAREHOUSE, 
    table_name='PLATFORM_AVERAGE', 
    loadmode="overwrite"
) 

✅ Successfully overwritten 435818 rows into FOLLOWERINSIGHTS_ANALYTICS.BI.PLATFORM_AVERAGE (1 chunk(s))



###### 3. Market Key (NEW!)

In [ ]:
write_df_to_snowflake(
    df=market_key_final, 
    database=WAREHOUSE, 
    table_name='Market_Key', 
    loadmode="overwrite"
) 

✅ Successfully overwritten 1224 rows into FOLLOWERINSIGHTS_ANALYTICS.BI.Market_Key (1 chunk(s))


In [ ]:
write_df_to_snowflake(
    df=market_key_V2, 
    database=WAREHOUSE, 
    table_name='MARKET', 
    loadmode="overwrite"
) 

✅ Successfully overwritten 584 rows into FOLLOWERINSIGHTS_ANALYTICS.BI.MARKET (1 chunk(s))
